<a href="https://colab.research.google.com/github/NguyenPhamQuocHuy-20130283/20130283_NguyenPhamQuocHuy_ML2023/blob/main/Lab_7_20130283_NguyenPhamQuocHuy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This lab deals with **cross validation** for some classification algorithms and **clustering methods**. 

*   **Deadline: 23:59, 10/4/2023**



# Import libraries

In [26]:
# code
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score 
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from prettytable import PrettyTable
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import CategoricalNB, BernoulliNB, ComplementNB, GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, RFE, SelectFromModel
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/Colab Notebooks'
     

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks


#Task 1. With **iris** dataset
*  1.1. Apply **SVM** algorithm with cross validation


In [8]:
#code
x = datasets.load_iris().data
y = datasets.load_iris().target

clf = svm.SVC(kernel="sigmoid", random_state=0)
scores = cross_validate(clf, x, y, scoring='accuracy', cv=10)

print(sorted(scores.keys()))
print(np.mean(scores['test_score']))





['fit_time', 'score_time', 'test_score']
0.06666666666666668


*  1.2 Apply **feature selection** to the dataset and then use **RandomForest** algorithm with cross validation 

In [10]:
#code
x_new = SelectKBest(chi2, k='all').fit_transform(x, y)
x_train, x_test, y_train, y_test = train_test_split(x_new, y, test_size=0.3)

rf = RandomForestClassifier(n_estimators=100)
rf_scores = cross_validate(rf, x_train, y_train, scoring="accuracy", cv=10);

print(sorted(rf_scores.keys()))
print(np.mean(rf_scores['test_score']))


['fit_time', 'score_time', 'test_score']
0.9354545454545455


#Task 2. With mnist dataset: 
*   2.1. Apply **K-Means** algorithm using k=10, 

In [36]:
from scipy.stats import mode
# loading dataset
dataset = datasets.load_iris()
# building model
x_train, x_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.3)
kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(x_train, y_train)

kmean_y = kmeans.predict(x_train)



print(accuracy_score(y_train, kmean_y))

0.10476190476190476


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


*   2.2. Compare the obtained result with with other classification algorithms such as **Randomforest**, **kNN**, and **Naïve Bayes** in terms of accuracy, precision, recall, f1 using cross validation. 


In [52]:
from numpy.lib.function_base import average
# code
dataset = datasets.load_iris()
x_train, x_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.3)
rf = RandomForestClassifier(n_estimators=100)
rf.fit(x_train, y_train)
rf_pred = rf.predict(x_test)

# Naive bayes
gauss = GaussianNB()
gauss.fit(x_train, y_train)
gauss_pred = gauss.predict(x_test)

knn = KNeighborsClassifier(n_neighbors=30)
knn.fit(x_train, y_train)
knn_pred = knn.predict(x_test);

kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(x_train, y_train)

kmean_y = kmeans.predict(x_test)

t = PrettyTable(["Algorithm","Accuracy","Precision", "Recall", "F1"])
t.add_row(["KNN", round(accuracy_score(y_test, knn_pred), 2), 
           round(precision_score(y_test, knn_pred, average='micro'), 2), 
           round(recall_score(y_test, knn_pred, average='macro'), 2),
           round(f1_score(y_test, knn_pred,average='macro'), 2)])
t.add_row(["Random forest", round(accuracy_score(y_test, rf_pred), 2), 
           round(precision_score(y_test, rf_pred, average='micro'), 2), 
           round(recall_score(y_test, rf_pred, average='macro'), 2),
           round(f1_score(y_test, rf_pred,average='macro'), 2)])
t.add_row(["Naive Bayes", round(accuracy_score(y_test, gauss_pred), 2), 
           round(precision_score(y_test, gauss_pred, average='micro'), 2), 
           round(recall_score(y_test, gauss_pred, average='macro'), 2),
           round(f1_score(y_test, gauss_pred,average='macro'), 2)])
t.add_row(["K Means", round(accuracy_score(y_test, kmean_y), 2), 
           round(precision_score(y_test, kmean_y, average='micro'), 2), 
           round(recall_score(y_test, kmean_y, average='macro'), 2),
           round(f1_score(y_test, kmean_y,average='macro'), 2)])
print(t)

+---------------+----------+-----------+--------+------+
|   Algorithm   | Accuracy | Precision | Recall |  F1  |
+---------------+----------+-----------+--------+------+
|      KNN      |   0.91   |    0.91   |  0.92  | 0.92 |
| Random forest |   0.96   |    0.96   |  0.96  | 0.96 |
|  Naive Bayes  |   0.96   |    0.96   |  0.96  | 0.96 |
|    K Means    |   0.02   |    0.02   |  0.01  | 0.01 |
+---------------+----------+-----------+--------+------+


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


*   2.3. From the obtained results, **which approach is better** for this problem: Supervised learning or Unsupervised learning?

In [ ]:
#code

2.4. Apply **AgglomerativeClustering** algorithm to mnist dataset using the number of clusters is 10

#Task 3. 
For given dataset (shopping-data.csv) including 5 attributes: **CustomerID**, **Genre**, **Age**, **Annual Income**, and **Spending Score**.
*   3.1. Using the **scipy library** to create the dendrograms for the given dataset (remember drop categorical attributes: **CustomerID**, **Genre**)





In [ ]:
#code

*   3.2. Apply K-Means to the preprocessed dataset with k belongs to [2,10]. Then compute SSE values and plot them to find the best value of k.

In [ ]:
#code

*   3.2. From the obtained dengrograms, choose an appropriate number of clusters and apply **AgglomerativeClustering** algorithm to the given dataset

In [ ]:
#code

#Finally,
Save a copy in your Github. Remember renaming the notebook.